In [2]:
import tensorflow as tf

In [3]:
tf.enable_eager_execution()

In [4]:
import tensorflow_datasets as tfds

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [5]:
imdb,info = tfds.load("imdb_reviews",with_info=True,as_supervised=True)

In [6]:
import numpy as np

In [7]:
train_data , test_data = imdb['train'],imdb['test']

In [8]:
train_sentences = []
train_labels = []
test_sentences = []
test_labels = []

In [9]:
for s,l in train_data:
    train_sentences.append(str(s.numpy()))
    train_labels.append(l.numpy())

In [10]:
for s,l in test_data:
    test_sentences.append(str(s.numpy()))
    test_labels.append(l.numpy())

In [11]:
train_labels_final = np.array(train_labels)
test_labels_final = np.array(test_labels)

# Tokenization

In [12]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [90]:
tokenizer = Tokenizer(num_words=10000,oov_token="<OOV>")

In [91]:
tokenizer.fit_on_texts(train_sentences)

In [92]:
word_index = tokenizer.index_word

In [93]:
word_index

{1: '<OOV>',
 2: 'the',
 3: 'and',
 4: 'a',
 5: 'of',
 6: 'to',
 7: 'is',
 8: 'br',
 9: 'in',
 10: 'it',
 11: 'i',
 12: 'this',
 13: 'that',
 14: 'was',
 15: 'as',
 16: 'for',
 17: 'with',
 18: 'movie',
 19: 'but',
 20: 'film',
 21: "'s",
 22: 'on',
 23: 'you',
 24: 'not',
 25: 'are',
 26: 'his',
 27: 'he',
 28: 'have',
 29: 'be',
 30: 'one',
 31: 'all',
 32: 'at',
 33: 'by',
 34: 'they',
 35: 'an',
 36: 'who',
 37: 'so',
 38: 'from',
 39: 'like',
 40: 'her',
 41: "'t",
 42: 'or',
 43: 'just',
 44: 'there',
 45: 'about',
 46: 'out',
 47: "'",
 48: 'has',
 49: 'if',
 50: 'some',
 51: 'what',
 52: 'good',
 53: 'more',
 54: 'very',
 55: 'when',
 56: 'she',
 57: 'up',
 58: 'can',
 59: 'b',
 60: 'time',
 61: 'no',
 62: 'even',
 63: 'my',
 64: 'would',
 65: 'which',
 66: 'story',
 67: 'only',
 68: 'really',
 69: 'see',
 70: 'their',
 71: 'had',
 72: 'were',
 73: 'me',
 74: 'well',
 75: 'we',
 76: 'than',
 77: 'much',
 78: 'been',
 79: 'get',
 80: 'bad',
 81: 'will',
 82: 'people',
 83: 'do',

In [94]:
train_seq = tokenizer.texts_to_sequences(train_sentences)

In [95]:
train_padded = pad_sequences(train_seq , maxlen=120 , truncating='post')

In [127]:
len(list(train_padded))

25000

In [117]:
train_padded

array([[  59,  474, 1281, ...,  547,   37,  112],
       [  59,   12,    7, ...,   20,   10,    7],
       [  59,   11,  439, ...,    2,  450, 1423],
       ...,
       [6162,   11,  707, ...,    8, 2623,  324],
       [   0,    0,    0, ...,   30,    3,   31],
       [8257,  147,   41, ...,    6,   29, 4022]])

In [96]:
test_seq = tokenizer.texts_to_sequences(test_sentences)

In [250]:
test_sentences[0]

'b"I\'ve watched the movie actually several times. And what i want to say about it is the only thing that made this movie high rank was the Burak Altay\'s incredible performance, absolutely nothing but that. Not even those silly model named Deniz Akkaya and some of these popular names at times in the movie... Burak is definitely very talented i\'ve seen a few jobs he made and been through. Even though this is kind of horror movie, he\'s doing really good job in comedy movies and also in dramas too. I bet most of you all saw Asmali Konak the movie and TV series, those two would go for an example... All i\'m gonna say is you better watch out for the new works coming out from Burak then you\'ll see.. Keep the good work bro, much love.."'

In [97]:
test_padded = pad_sequences(test_seq , maxlen=120)

In [98]:
from tensorflow.keras.models import Sequential

In [99]:
from tensorflow.keras.layers import Dense , Embedding , Flatten ,GlobalAvgPool1D

In [199]:
model = Sequential()

In [200]:
model.add(Embedding(10000 , 17 ,input_length = 120))
model.add(GlobalAvgPool1D())
model.add(Dense(3 , activation='relu'))
model.add(Dense(1 , activation='sigmoid'))

In [201]:
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 120, 17)           170000    
_________________________________________________________________
global_average_pooling1d_19  (None, 17)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 3)                 54        
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 4         
Total params: 170,058
Trainable params: 170,058
Non-trainable params: 0
_________________________________________________________________


In [202]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
model.summary()

Model: "sequential_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_20 (Embedding)     (None, 120, 17)           170000    
_________________________________________________________________
global_average_pooling1d_19  (None, 17)                0         
_________________________________________________________________
dense_45 (Dense)             (None, 3)                 54        
_________________________________________________________________
dense_46 (Dense)             (None, 1)                 4         
Total params: 170,058
Trainable params: 170,058
Non-trainable params: 0
_________________________________________________________________


In [203]:
model.fit(train_padded,train_labels_final,epochs=10,validation_data=(test_padded,test_labels_final))

Train on 25000 samples, validate on 25000 samples
Epoch 1/10
25000/25000 [==============================] - 9s 352us/sample - loss: 0.5968 - acc: 0.7379 - val_loss: 0.4455 - val_acc: 0.8315
Epoch 2/10
25000/25000 [==============================] - 9s 344us/sample - loss: 0.3731 - acc: 0.8522 - val_loss: 0.3467 - val_acc: 0.8531
Epoch 3/10
25000/25000 [==============================] - 8s 301us/sample - loss: 0.2990 - acc: 0.8782 - val_loss: 0.3281 - val_acc: 0.8594
Epoch 4/10
25000/25000 [==============================] - 7s 292us/sample - loss: 0.2604 - acc: 0.8964 - val_loss: 0.3293 - val_acc: 0.8585
Epoch 5/10
25000/25000 [==============================] - 7s 290us/sample - loss: 0.2338 - acc: 0.9099 - val_loss: 0.3392 - val_acc: 0.8559
Epoch 6/10
25000/25000 [==============================] - 7s 291us/sample - loss: 0.2128 - acc: 0.9197 - val_loss: 0.3525 - val_acc: 0.8520
Epoch 7/10
25000/25000 [==============================] - 8s 310us/sample - loss: 0.1951 - acc: 0.9279 - val_l

In [204]:
e = model.layers[0]
weights = e.get_weights()[0]
print(weights.shape)

(10000, 17)


In [183]:
import io

In [212]:
word_index[1]

'<OOV>'

In [225]:
out_v = io.open('vecs.tsv','w',encoding='utf-8')
out_m = io.open('meta.tsv','w',encoding='utf-8')
for word_num in range(1,10000):
    word = word_index[word_num]
    embeddings = weights[word_num]
    out_m.write(word + "\n")
    out_v.write('\t'.join([str(x) for x in embeddings]) +"\n")
out_v.close()
out_m.close()